In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
# import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
#Read data from csv
train = pd.read_csv('../1_Data/Restate_int_train.csv')
test = pd.read_csv('../1_Data/Restate_int_test.csv')
external = pd.read_csv('../1_Data/WRDS.csv') #external data
data_dict = pd.read_excel('../1_Data/DataDictionary_v2.xlsx') #list of features that we are going to use 

In [4]:
data_dict.head()

,Formula,Name,Account
0,gvkey,Global Company Key,NaN
1,fyear,Data Year - Fiscal,NaN
2,acominc,Accumulated Other Comprehensive Income (Loss),Comprehensive Income
3,aedi,Accrued Expenses and Deferred Income,Accrued Expenses
4,act,Current Assets - Total,Assets


# 1. Exploratory Data Analysis

## 1.1 Find percentage of missing data

In [5]:
data_dict.head()

,Formula,Name,Account
0,gvkey,Global Company Key,NaN
1,fyear,Data Year - Fiscal,NaN
2,acominc,Accumulated Other Comprehensive Income (Loss),Comprehensive Income
3,aedi,Accrued Expenses and Deferred Income,Accrued Expenses
4,act,Current Assets - Total,Assets


In [6]:
external.head()

,GVKEY,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,acominc,...,auop,auopic,conml,ggroup,gind,gsector,gsubind,sic,spcsrc,ipodate
0,1004,20050531,2004,INDL,C,D,STD,AAR CORP,USD,-19.779,...,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
1,1004,20060531,2005,INDL,C,D,STD,AAR CORP,USD,-13.842,...,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
2,1004,20070531,2006,INDL,C,D,STD,AAR CORP,USD,-13.899,...,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
3,1004,20080531,2007,INDL,C,D,STD,AAR CORP,USD,-13.012,...,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
4,1004,20090531,2008,INDL,C,D,STD,AAR CORP,USD,-23.996,...,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0


In [7]:
#Now we only take the list of features from the DataDictionary_v2.xlsx
feature_list = data_dict['Formula'].values.tolist() #create a list of features
external.rename(columns={'GVKEY': 'gvkey'}, inplace=True)
external = external[feature_list] #filter out
external.rename(columns={'fyear': 'year'}, inplace=True)

#change column name into gvkey and year 


In [8]:

def find_missing(data):
    cnt_missing = data.isnull().sum().values #number of missing values
    total = data.shape[0] # total records
    ratio_missing = cnt_missing/total*100 #percentage of missing
    
    return pd.DataFrame(data ={'missing_count': cnt_missing, 'missing_ratio_percentage' : ratio_missing}, index = data.columns.values)
df_missing = find_missing(external)
df_missing.style.set_properties(**{'text-align': 'left'})
# df_missing.sort_values(['missing_count','missing_ratio_percentage'],ascending = False)

,missing_count,missing_ratio_percentage
gvkey,0,0
year,0,0
acominc,3985,11.2269
aedi,35495,100
act,10504,29.5929
am,11745,33.0892
ap,4005,11.2833
aqc,4835,13.6216
artfs,35495,100
at,3761,10.5959


In [9]:
#We only take 
df_missing.reset_index(drop= False,inplace = True)
ext_feature = df_missing[df_missing['missing_ratio_percentage']<15]['index'].values #take feature with less than 15% missing values 

In [10]:
#then we are using these feature to merge with our train and test data 
external = external[ext_feature] #filter out these feature
external = external.select_dtypes(exclude=['object']) #for now we dont take categorical features 
train = train.merge(right = external,on =['gvkey','year'],how= 'left')
test = test.merge(right = external,on =['gvkey','year'],how= 'left')

In [11]:
train.shape

(15213, 45)

In [12]:
#convert to date time
train['Date'] =  pd.to_datetime(train['Date'])
test['Date'] =  pd.to_datetime(test['Date'])

train.head()

,gvkey,year,Filing,Date,Restate_Int,acominc,ap,aqc,at,bkvlps,...,re,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic
0,1004,2005,0001104659-05-033688,2005-07-22,0,-13.842,97.002,0.000,978.819,11.5326,...,183.550,136.272,897.284,422.717,69.664,0.0,0.0,6.0,1.0,1.0
1,1004,2006,0001104659-06-047248,2006-07-17,0,-13.899,110.239,38.478,1067.633,13.0998,...,242.153,181.691,1061.169,494.243,79.813,0.0,0.0,6.0,1.0,1.0
2,1004,2007,0001104659-07-055173,2007-07-20,0,-13.012,99.073,85.210,1362.010,15.0944,...,318.184,202.472,1384.919,585.255,100.935,0.0,0.0,6.0,1.0,1.0
3,1004,2008,0001047469-08-008126,2008-07-11,0,-23.996,100.651,0.000,1377.511,16.8937,...,385.851,227.300,1423.976,656.895,103.159,0.0,0.0,6.0,1.0,1.0
4,1004,2009,0001047469-09-006783,2009-07-16,0,-29.646,114.906,193.989,1501.042,18.9167,...,389.641,238.466,1352.151,746.906,104.447,0.0,0.0,6.0,4.0,1.0


In [13]:
train['Restate_Int'].value_counts()

0    14850
1      363
Name: Restate_Int, dtype: int64

In [14]:
#sort by gvkey and year
train.sort_values(['gvkey','year'],inplace = True)
test.sort_values(['gvkey','year'],inplace = True)

In [15]:
train['year'].value_counts()

2005    3112
2009    3111
2006    3017
2008    2998
2007    2975
Name: year, dtype: int64

# 2.Feature engineering

In [16]:
train['Weekday'] = train['Date'].dt.dayofweek
test['Weekday'] = test['Date'].dt.dayofweek
test.head()

,gvkey,year,Filing,Date,acominc,ap,aqc,at,bkvlps,capx,...,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,Weekday
0,1004,2010,0001047469-10-006500,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,1045,2010,0000006201-10-000006,2010-02-17,-2755.000,1156.000,0.0,25088.000,-11.8309,1962.000,...,738.000,22170.000,-3945.000,367.000,0.0,0.0,4.0,1.0,1.0,2
2,1050,2010,0001193125-10-069639,2010-03-29,-1.608,9.712,0.0,74.791,2.4565,0.654,...,26.772,140.602,35.174,0.356,0.0,0.0,11.0,1.0,0.0,0
3,1072,2010,0000859163-10-000020,2010-05-20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,1075,2010,0000950123-10-014366,2010-02-19,-159.767,236.354,0.0,12362.703,33.8634,765.152,...,419.518,3263.645,3683.327,2.239,2.1,2.1,5.0,4.0,1.0,4


In [17]:
train['Date_lag'] = train.groupby('gvkey')['Date'].shift(1)
test['Date_lag'] = test.groupby('gvkey')['Date'].shift(1)
train['Date_diff'] = (train['Date'] -train['Date_lag']).dt.days -365
test['Date_diff'] = (test['Date'] -test['Date_lag']).dt.days - 365

In [18]:
train.head()

,gvkey,year,Filing,Date,Restate_Int,acominc,ap,aqc,at,bkvlps,...,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,Weekday,Date_lag,Date_diff
0,1004,2005,0001104659-05-033688,2005-07-22,0,-13.842,97.002,0.000,978.819,11.5326,...,422.717,69.664,0.0,0.0,6.0,1.0,1.0,4,NaT,NaN
1,1004,2006,0001104659-06-047248,2006-07-17,0,-13.899,110.239,38.478,1067.633,13.0998,...,494.243,79.813,0.0,0.0,6.0,1.0,1.0,0,2005-07-22,-5.0
2,1004,2007,0001104659-07-055173,2007-07-20,0,-13.012,99.073,85.210,1362.010,15.0944,...,585.255,100.935,0.0,0.0,6.0,1.0,1.0,4,2006-07-17,3.0
3,1004,2008,0001047469-08-008126,2008-07-11,0,-23.996,100.651,0.000,1377.511,16.8937,...,656.895,103.159,0.0,0.0,6.0,1.0,1.0,4,2007-07-20,-8.0
4,1004,2009,0001047469-09-006783,2009-07-16,0,-29.646,114.906,193.989,1501.042,18.9167,...,746.906,104.447,0.0,0.0,6.0,4.0,1.0,3,2008-07-11,5.0


In [19]:
train['Week_num'] =train['Date'].dt.week#.dt.date
test['Week_num'] =test['Date'].dt.week#.dt.date
train.head()

,gvkey,year,Filing,Date,Restate_Int,acominc,ap,aqc,at,bkvlps,...,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,Weekday,Date_lag,Date_diff,Week_num
0,1004,2005,0001104659-05-033688,2005-07-22,0,-13.842,97.002,0.000,978.819,11.5326,...,69.664,0.0,0.0,6.0,1.0,1.0,4,NaT,NaN,29
1,1004,2006,0001104659-06-047248,2006-07-17,0,-13.899,110.239,38.478,1067.633,13.0998,...,79.813,0.0,0.0,6.0,1.0,1.0,0,2005-07-22,-5.0,29
2,1004,2007,0001104659-07-055173,2007-07-20,0,-13.012,99.073,85.210,1362.010,15.0944,...,100.935,0.0,0.0,6.0,1.0,1.0,4,2006-07-17,3.0,29
3,1004,2008,0001047469-08-008126,2008-07-11,0,-23.996,100.651,0.000,1377.511,16.8937,...,103.159,0.0,0.0,6.0,1.0,1.0,4,2007-07-20,-8.0,28
4,1004,2009,0001047469-09-006783,2009-07-16,0,-29.646,114.906,193.989,1501.042,18.9167,...,104.447,0.0,0.0,6.0,4.0,1.0,3,2008-07-11,5.0,29


In [20]:
train.to_csv('../1_Data/Metadata/train.csv')
test.to_csv('../1_Data/Metadata/test.csv')